In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
root_data_dir = "../data"

# Exercise 1 Mean & Std deviation of CIFAR datasets (Before and after Normalization)
## Exercise 1.1: Calculate it on the Dataset (not on the data loader)
Notice that there is no change before and after normalization. This is because the transform (transforms.toTensor and the Normalization) are not applied on the dataset. This only takes effect after the dataloader

In [ ]:

# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper parameters
num_epochs = 10
batch_size = 32
learning_rate = 0.001

# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset1 = torchvision.datasets.CIFAR10(root=root_data_dir, train=True,
                                        download=True, transform=transforms.ToTensor())

test_dataset1 = torchvision.datasets.CIFAR10(root=root_data_dir, train=False,
                                       download=True, transform=transforms.ToTensor())
# CIFAR 10
print("BEFORE TRANSFORMATION(Before Normalization): Data mean and std on the DATASET")
print("train_dataset: details (before transformation")
print(f'train_dataset1 size     :{list(train_dataset1.data.shape)}')
print(f'train_dataset1 mean     :{train_dataset1.data.mean(axis = (0,1,2))}')
print(f'train_dataset1 mean/255 :{train_dataset1.data.mean(axis = (0,1,2))/255}') 
print(f'train_dataset1 std-dev  :{train_dataset1.data.std(axis = (0,1,2))}')
print(f'train_dataset1 std-dev/255:{train_dataset1.data.std(axis = (0,1,2))/255}') 

print("\ntest_dataset: details (before transformation")
print(f'test_dataset1 size     :{list(test_dataset1.data.shape)}')
print(f'test_dataset1 mean     :{test_dataset1.data.mean(axis = (0,1,2))}')
print(f'test_dataset1 mean/255 :{test_dataset1.data.mean(axis = (0,1,2))/255}') 
print(f'test_dataset1 std-dev  :{test_dataset1.data.std(axis = (0,1,2))}')
print(f'test_dataset1 std-dev/255:{test_dataset1.data.std(axis = (0,1,2))/255}') 

# dataset has PILImage images of range [0.1]
# We transform them to i) Tensors ii) normalized in the range [0,1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean= (125.30691805,122.95039414,113.86538318), # These normalization values are wrong, more on that later
                          std=(62.99321928,62.08870764,66.70489964))])
train_dataset = torchvision.datasets.CIFAR10(root=root_data_dir, train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root=root_data_dir, train=False,
                                       download=True, transform=transform)


# CIFAR 10
print("\n\nAFTER TRANSFORMATION(After Normalization): Data mean and std on the DATASET (not dataloader) \
       \nNotice that there is no change. \
       \nThis is because the  Transforms (and hence the normalization) does not take effect until the data loader is called")
print("train_dataset: details (after transformation)")
print(f'train_dataset size     :{list(train_dataset.data.shape)}')
print(f'train_dataset mean     :{train_dataset.data.mean(axis = (0,1,2))}')
print(f'train_dataset mean/255 :{train_dataset.data.mean(axis = (0,1,2))/255}') 
print(f'train_dataset std-dev  :{train_dataset.data.std(axis = (0,1,2))}')
print(f'train_dataset std-dev/255:{train_dataset.data.std(axis = (0,1,2))/255}') 

print("\ntest_dataset: details (after transformation")
print(f'test_dataset size     :{list(test_dataset.data.shape)}')
print(f'test_dataset mean     :{test_dataset.data.mean(axis = (0,1,2))}')
print(f'test_dataset mean/255 :{test_dataset.data.mean(axis = (0,1,2))/255}') 
print(f'test_dataset std-dev  :{test_dataset.data.std(axis = (0,1,2))}')
print(f'test_dataset std-dev/255:{test_dataset.data.std(axis = (0,1,2))/255}') 

# Exercie 1: Mean & Std deviation of CIFAR datasets (Before and after Normalization)
## Exercise 1.2: Calculate it on the DATALOADER (not on the dataset)


### Transforms and Dataloader
In PyTorch, transforms are applied to data when the data is accessed through a DataLoader. Specifically, the transforms specified in the Dataset are applied within the __getitem__ method of the Dataset class. When a DataLoader iterates through the Dataset, it calls __getitem__ for each index, and it is at this point that the transforms are applied to the data before it is returned.
If you are using a custom Dataset, you would typically include the transform application within your __getitem__ method:
Python

Hence the effect of Normalization can only be seen when the mean and standard deviation are calculated on the dataloader


In [ ]:

# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset1 = torchvision.datasets.CIFAR10(root=root_data_dir, 
                                              train=True,
                                              download=True,
                                              transform=transforms.ToTensor())

test_dataset1 = torchvision.datasets.CIFAR10(root=root_data_dir,
                                             train=False,
                                             download=True, transform=transforms.ToTensor())
# Create a dataloader
trainloader1 = torch.utils.data.DataLoader(train_dataset1, batch_size=100, shuffle=False)

# Calculate the mean and standard deviation
mean = 0
std = 0
for i, data in enumerate(trainloader1, 0):
    # Get the input data
    inputs, labels = data

    # Calculate the mean and standard deviation for each batch
    mean += torch.mean(inputs, dim=[0, 2, 3])
    std += torch.std(inputs, dim=[0, 2, 3])

# Calculate the overall mean and standard deviation
mean /= i + 1
std /= i + 1

print("Mean(before normalization):", mean)
print("Standard Deviation(before normalization):", std)

# dataset has PILImage images of range [0.1]
# We transform them to i) Tensors ii) normalized in the range [0,1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean= (125.30691805,122.95039414,113.86538318), # These normalization values are wrong, more on that later
                          std=(62.99321928,62.08870764,66.70489964))])
train_dataset = torchvision.datasets.CIFAR10(root=root_data_dir, train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root=root_data_dir, train=False,
                                       download=True, transform=transform)


# Create a dataloader
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=False)

# Calculate the mean and standard deviation
mean = 0
std = 0
for i, data in enumerate(trainloader, 0):
    # Get the input data
    inputs, labels = data

    # Calculate the mean and standard deviation for each batch
    mean += torch.mean(inputs, dim=[0, 2, 3])
    std += torch.std(inputs, dim=[0, 2, 3])

# Calculate the overall mean and standard deviation
mean /= i + 1
std /= i + 1

print("\n Mean(after normalization):\n", mean)
print("Standard Deviation(after normalization: \n the normalization values used could be wrong !!!!!. More on that later):\n", std)

# Exercise 2 : Transforms, Mean and Standard Deviation
https://stackoverflow.com/questions/66678052/how-to-calculate-the-mean-and-the-std-of-cifar10-data \


## Exercise 2.1: Just using Min and Max. No dataloader, no inline iteration , no for loop
Notice that in the below example you have to divide the dataset by 255 in order to get the same results as section 2.2, 2.3 and 2.4
This is because in sections 2.2- 2.4, the transorm= transforms.ToTensor() is applied (while this is not applied in section 2.1)

### What does transforms.ToTensor() do
https://pytorch.org/vision/main/generated/torchvision.transforms.ToTensor.html \
transforms.ToTensor assumes that a PIL Image in the range [0,255] is being passed. It divides the image by 255 and normalizes in the range [0,1] \
Note: \
1) the max value in the dataset could be 200 and not 255, in which case the max value in the dataset would be 200/255 = 0.7843. So another subsequent normalization might be necessary to truly make the range of the data [0,1] (and this entire notebook is about that LOL) \
2) When composing transforms. the order is i) transforms.ToTensor ii) Normalization \
   Example: transform = transforms.Compose( \
   [transforms.ToTensor(), \
   transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),\
                        std=(0.2466, 0.2431, 0.2610))])



In [ ]:
from  torchvision import datasets

cifar_trainset = datasets.CIFAR10(root=root_data_dir, train=True, download=True  )
data = cifar_trainset.data / 255 # data is numpy array

mean  = cifar_trainset.data.mean(axis = (0,1,2)) 
std   = cifar_trainset.data.std(axis = (0,1,2))
print("Mean and Standard Deviation before dividing by 255")
print("Mean:", mean)
print("STD :", std)

mean = data.mean(axis = (0,1,2)) 
std  = data.std(axis = (0,1,2))
 #Mean : [0.491 0.482 0.446]   STD: [0.247 0.243 0.261]
print("\nMean and Standard Deviation After dividing by 255")
print("Mean:", mean)
print("STD :", std)

# Exercise 2 : Transforms, Mean and Standard Deviation
https://stackoverflow.com/questions/66678052/how-to-calculate-the-mean-and-the-std-of-cifar10-data \
## Exercise 2.2 : Without an explicit data loader: For Loop
Notice that in the below example there is no explicit data loader despite that the transorm= transforms.ToTensor() is applied

This is because when iterating over a PyTorch Dataset in a for loop or using it with a DataLoader, the __getitem__ method is called for each index accessed during the iteration. This is how PyTorch retrieves individual data samples from the dataset.\



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root=root_data_dir, train=True, download=True, transform=transforms.ToTensor())

# Initialize variables for mean and standard deviation calculation
total_sum = torch.zeros(3)
total_squared_sum = torch.zeros(3)
num_batches = len(trainset)
count = 0

# Iterate through the dataset to calculate the sum of pixel values and the sum of squared pixel values
for data, _ in trainset:
    total_sum += torch.sum(data, dim=[1, 2])
    total_squared_sum += torch.sum(data ** 2, dim=[1, 2])
    count += data.shape[1] * data.shape[2]

# Calculate the mean for each channel
mean = total_sum / count

# Calculate the standard deviation for each channel
std = torch.sqrt((total_squared_sum / count) - (mean ** 2))

print("Mean and Standard Deviation Using For Loop")
print("Mean:", mean)
print("STD :", std)

# Exercise 2 : Transforms, Mean and Standard Deviation
https://stackoverflow.com/questions/66678052/how-to-calculate-the-mean-and-the-std-of-cifar10-data \
## Exercise 2.3 : Without an explicit data loader: Inline iteration
Similar to for loop
Notice that in the below example there is no explicit data loader despite that the transorm= transforms.ToTensor() is applied

This is because when you iterate directly over a Dataset object, Python implicitly calls the __iter__ method, which then relies on __getitem__ to fetch the data. The iteration continues until an IndexError is raised, signaling the end of the dataset. \


In [ ]:
import torch
import numpy
import torchvision.datasets as datasets
from torchvision import transforms

cifar_trainset = datasets.CIFAR10(root=root_data_dir, train=True, download=True, transform=transforms.ToTensor())

imgs = [item[0] for item in cifar_trainset] # item[0] and item[1] are image and its label
imgs = torch.stack(imgs, dim=0).numpy()

# calculate mean over each channel (r,g,b)
mean_r = imgs[:,0,:,:].mean()
mean_g = imgs[:,1,:,:].mean()
mean_b = imgs[:,2,:,:].mean()

# calculate std over each channel (r,g,b)
std_r = imgs[:,0,:,:].std()
std_g = imgs[:,1,:,:].std()
std_b = imgs[:,2,:,:].std()

print("Mean and Standard Deviation: Inline Iteration")
print("Mean :",mean_r,mean_g,mean_b)
print("STD  :", std_r,std_g,std_b)

# Exercise 2 : Transforms, Mean and Standard Deviation
https://stackoverflow.com/questions/66678052/how-to-calculate-the-mean-and-the-std-of-cifar10-data \
## Exercise 2.4 : With a data loader
### Difference between for loop /iteration vs dataloader for loading data
#### For Loop Iteration 
A basic for loop directly accesses elements from the dataset one by one. This approach is straightforward for small datasets but becomes inefficient for large datasets due to the lack of parallel processing and batching.\

#### DataLoader Iteration
When using a DataLoader, it handles the iteration process, including batching, shuffling, and parallel loading. This significantly improves efficiency, especially for large datasets, by utilizing multiple CPU cores and reducing data loading time. The dataloader internally calls __getitem__ to retrieve the data samples based on the sampler it uses. The dataloader uses a Sampler to generate indices, and for each index, the __getitem__ method is called to retrieve the corresponding data sample. This happens for every item in the dataset during a full iteration

### Exercise 2.4.1: How NOT TO use a dataloader !!!! Lol !

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Load the CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root=root_data_dir, train=True, download=True, transform=transforms.ToTensor())

# Create a dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)

def get_mean_std(trainLoader):
    imgs = None
    for batch in trainLoader:
        image_batch = batch[0]
        if imgs is None:
            imgs = image_batch.cpu()
        else:
            imgs = torch.cat([imgs, image_batch.cpu()], dim=0)
    imgs = imgs.numpy()
    
    # calculate mean over each channel (r,g,b)
    mean_r = imgs[:,0,:,:].mean()
    mean_g = imgs[:,1,:,:].mean()
    mean_b = imgs[:,2,:,:].mean()

    # calculate std over each channel (r,g,b)
    std_r = imgs[:,0,:,:].std()
    std_g = imgs[:,1,:,:].std()
    std_b = imgs[:,2,:,:].std()


    print("Mean and Standard Deviation: Data Loader ")
    print("This is so slow , dont use this method")
    print("Mean :",mean_r,mean_g,mean_b)
    print("STD  :", std_r,std_g,std_b)

get_mean_std(train_loader)


In [ ]:
### Exercise 2.4.2: Correct way TO use a dataloader !!!! Lol !

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the transform to convert images to tensors
transform = transforms.ToTensor()

# Load the CIFAR10 training dataset
trainset = torchvision.datasets.CIFAR10(root=root_data_dir, train=True, download=True, transform=transform)

# Create a DataLoader
dataloader = DataLoader(trainset, batch_size=10000, shuffle=False, num_workers=2)

# Initialize variables to store the sum and squared sum of pixel values
channels_sum, channels_squared_sum = 0, 0
num_batches = 0

# Iterate through the DataLoader to calculate the sum and squared sum
for data, _ in dataloader:
    channels_sum += torch.mean(data, dim=[0, 2, 3])
    channels_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
    num_batches += 1
    
# Calculate the mean and standard deviation
mean = channels_sum / num_batches
std = (channels_squared_sum / num_batches - mean**2) ** 0.5

print("Mean and Standard Deviation: Data Loader : CORRECT METHOD")
print(f"Mean: {mean}")
print(f"STD : {std}")